# On stress outputs from ASPECT
## Problem setup
When a Newtonian fluid subjected to one-dimensional compression and it has a uniformly constant viscosity and density, we can find the analytic solution. The compression is driven by inward velocity boundary conditions of $\pm$1 cm/yr on the boundaries at $x=0$ and $x=L$. Here $L$ is the length of the domain.
The top surface at $z=H$ is a free surface, where $H$ is the domain height, 400 km. All the other boundaries are free-slip surfaces. This problem is essentially two-dimensional but we construct it as a three-dimensional one for comparison with ASPECT.

## Velocity solution
With the given boundary conditions, the velocity field should be
\begin{eqnarray}
  &v_{x} &= \frac{v_{p}(L-2x)}{L}, \quad 0 \le x \le L,\\
  &v_{y} &= 0, \\
  &v_{z} &= \frac{2v_{p}z}{L}, \quad 0 \le z \le H.
\end{eqnarray}
The value of $v_{z}$ at $z=H$ is derived from the volume conservation (i.e., incompressibility): $v_{z}^{top}L = 2v_{p}H$.


## Strain rate and deviatoric stress
The only non-zero strain rate components are the following two:
\begin{eqnarray}
  &\varepsilon_{xx} &= -\frac{2v_{p}}{L}, \\
  &\varepsilon_{zz} &= \frac{2v_{p}}{L}.
\end{eqnarray}
The corresponding deviatoric stress components (called "shear_stress" in ASPECT) are
\begin{eqnarray}
  &\tau_{xx} &= -2\eta\frac{2v_{p}}{L}, \\
  &\tau_{zz} &= 2\eta\frac{2v_{p}}{L}.
\end{eqnarray}

Plugging in $L = 2000$ km, $v_{p} =$ 1 cm/yr $\approx$ 3.17$\times$10$^{-8}$ m/s and $\eta=10^{24}$ Pa$\cdot$s, we get the values of deviatric stresses as follows:
\begin{eqnarray}
  &\tau_{xx} &\approx -6.34\times 10^{8}, \\
  &\tau_{zz} &\approx 6.34\times 10^{8}.
\end{eqnarray}

In terms of **both sign and magnitude**, these deviatoric stresses are close to ASPECT's `shear_stress_xx` and `shear_stress_zz`. From this, we conclude that ASPECT's `shear_stress_??` outputs follow the "tension positive" convention as in our analytic solution.

In [1]:
vp = 0.01/3.1536e7
L = 2.0e6
eta = 1.0e24
tau = 4.0*eta*vp/L
print("{0:e}".format(tau))

6.341958e+08


## Pressure
Now, in order to calculate pressure, we need to consider the full momentum balance equation:
    \begin{eqnarray}
     & \frac{\partial p}{\partial x} - 2\eta \left[ \frac{\partial \varepsilon_{xx}}{\partial x} + \frac{\partial \varepsilon_{xy}}{\partial y} + \frac{\partial \varepsilon_{xz}}{\partial z} \right] &= 0, \\
     & \frac{\partial p}{\partial y} - 2\eta \left[ \frac{\partial \varepsilon_{yx}}{\partial x} + \frac{\partial \varepsilon_{yy}}{\partial y} + \frac{\partial \varepsilon_{yz}}{\partial z} \right] &= 0, \\
     & \frac{\partial p}{\partial z} - 2\eta \left[ \frac{\partial \varepsilon_{zx}}{\partial x} + \frac{\partial \varepsilon_{zy}}{\partial y} + \frac{\partial \varepsilon_{zz}}{\partial z} \right] &= -\rho g,
    \end{eqnarray}
where the minus sign for the body force term means that graviational acceleration has the negative $z$ direction.

The above set of equations are derived from ASPECT User Manual's equation (1) when viscosity and density are uniformly constant and the material is incompressible (i.e., $\nabla \cdot \mathbf{u}=0$).
Also note that the oppposite sign of pressure and velocity term implies that pressure follows the "compression positive" convention. These different sign conventions for deviatoric stress and pressure are consistent with the decomposition of the Cauchy stress ($\boldsymbol{\sigma}$) in the classical fluid/continuum mechanics: \begin{equation}
  \boldsymbol{\sigma} = -p \mathbf{I} + \boldsymbol{\tau}.
\end{equation}
So is our derivation of the solution.

Since even the only non-zero strain rate components are constant in space, all the partial derivatives of strain rate are zero in the above equations. Also, we note from the first two equations that pressure should be independent of $x$ and $y$ coordinates. The third equation gives
\begin{equation}
  p = -\rho\,g\,z + p_{0},
\end{equation}
where $p_{0}$ is a integration constant to be determined by an appropriate boundary condition like the top boundary being a free (i.e., traction-free) surface.

### $p=0$ on the top surface
Let's assume that if the top surface at $z = H$ is a free surface, pressure should be zero there. An alternative condition would be that $\boldsymbol{\sigma}\cdot\mathbf{n}=0$ at $z = H$. 
If pressure should be zero to have a free top surface, we get
\begin{equation}
  p = \rho\,g\,(H-z).
\end{equation}
Plugging in $\rho = 3300$ kg/m$^{3}$ and $g = 9.8$ m/s$^{2}$ above, we get
- $p=0$ at top ($z=H$)
- $p \approx 1.29\times10^{10}$ Pa at bottom ($z=0$).


In [6]:
rho = 3300.0; g = 9.8; H = 4e5
p0_1 = rho*g*H
print("p0_1:{0:e}".format(p0_1))

p0_1:1.293600e+10


### $\boldsymbol{\sigma}\cdot\mathbf{n}=0$ on the top surface
For this condition, we require $\sigma_{zz}(z=H) = -p + \tau_{zz} = \rho\,g\,H - p_{0} + \tau_{zz} = 0$. From this, $p_{0} = \rho g H + \tau_{zz}$. 
\begin{equation}
  p = \rho g (H-z) + \tau_{zz}.
\end{equation}
- $p = \tau_{zz} \approx 6.34 \times 10^{8}$ at top ($z=H$)
- $p \approx 1.36\times10^{10}$ Pa at bottom ($z=0$).


In [7]:
p0_2 = rho*g*H + tau
pH_2 = tau
print("p0_2:{0:e}, pH_2:{1:e}".format(p0_2, pH_2))

p0_2:1.357020e+10, pH_2:6.341958e+08


### $\sigma_{xx}=0$ on the top surface
For this condition, we require $\sigma_{xx}(z=H) = -p + \tau_{xx} = \rho\,g\,H - p_{0} + \tau_{xx} = 0$. From this, $p_{0} = \rho g H + \tau_{xx}$. 
\begin{equation}
  p = \rho g (H-z) + \tau_{xx}.
\end{equation}
- $p = \tau_{xx} \approx -6.34 \times 10^{8}$ at top ($z=H$)
- $p \approx 1.23\times10^{10}$ Pa at bottom ($z=0$).


In [8]:
p0_3 = rho*g*H - tau
print("p0_3:{0:e}".format(p0_3))

p0_3:1.230180e+10


## Cauchy stress 
Now we want to know the Cauchy stress $\mathbf{\sigma}$, which is decomposed into pressure and deviatoric part as follows:
\begin{equation}
  \boldsymbol{\sigma} = -p \mathbf{I} + \boldsymbol{\tau}.
\end{equation}


### $p=0$ on the top surface
At $z=0$,
\begin{eqnarray}
  &\sigma_{xx} &= -p + \tau_{xx} = -1.29 \times 10^{10} - 6.34 \times 10^{8} &= -1.35 \times 10^{10}, \\
  &\sigma_{yy} &= -p + \tau_{yy} = -1.29 \times 10^{10} + 0 &= -1.29 \times 10^{10}, \\
  &\sigma_{zz} &= -p + \tau_{zz} = -1.29 \times 10^{10} + 6.34 \times 10^{8} &= -1.23 \times 10^{10}, \\
  &\sigma_{ij} &= \tau_{ij} \quad \text{ if } i \neq j.
\end{eqnarray}

At $z=H$,
\begin{eqnarray}
  &\sigma_{xx} &= -p + \tau_{xx} = 0 - 6.34 \times 10^{8} &= -6.34 \times 10^{8}, \\
  &\sigma_{yy} &= -p + \tau_{yy} = 0 + 0 &= 0, \\
  &\sigma_{zz} &= -p + \tau_{zz} = 0 + 6.34 \times 10^{8} &= 6.34 \times 10^{8}.
\end{eqnarray}


### $\boldsymbol{\sigma}\cdot\mathbf{n}=0$ on the top surface
At the bottom ($z=0$),
\begin{eqnarray}
  &\sigma_{xx} &= -p + \tau_{xx} = -1.36 \times 10^{10} - 6.34 \times 10^{8} &= -1.42 \times 10^{10}, \\
  &\sigma_{yy} &= -p + \tau_{yy} = -1.36 \times 10^{10} + 0 &= -1.35 \times 10^{10}, \\
  &\sigma_{zz} &= -p + \tau_{zz} = -1.36 \times 10^{10} + 6.34 \times 10^{8} &= -1.29 \times 10^{10}.
\end{eqnarray}

At the top surface ($z=H$),
\begin{eqnarray}
  &\sigma_{xx} &= -p + \tau_{xx} = - 6.34 \times 10^{8} - 6.34 \times 10^{8} &= -1.27 \times 10^{9}, \\
  &\sigma_{yy} &= -p + \tau_{yy} = - 6.34 \times 10^{8} + 0 &= -6.34 \times 10^{8}, \\
  &\sigma_{zz} &= -p + \tau_{zz} = - 6.34 \times 10^{8} + 6.34 \times 10^{8} &= 0.
\end{eqnarray}



### $\sigma_{xx}=0$ on the top surface
At the bottom ($z=0$),
\begin{eqnarray}
  &\sigma_{xx} &= -p + \tau_{xx} = -1.23 \times 10^{10} - 6.34 \times 10^{8} &= -1.29 \times 10^{10}, \\
  &\sigma_{yy} &= -p + \tau_{yy} = -1.23 \times 10^{10} + 0 &= -1.23 \times 10^{10}, \\
  &\sigma_{zz} &= -p + \tau_{zz} = -1.23 \times 10^{10} + 6.34 \times 10^{8} &= -1.17 \times 10^{10}.
\end{eqnarray}

At the top surface ($z=H$),
\begin{eqnarray}
  &\sigma_{xx} &= -p + \tau_{xx} =  6.34 \times 10^{8} - 6.34 \times 10^{8} &= 0, \\
  &\sigma_{yy} &= -p + \tau_{yy} =  6.34 \times 10^{8} + 0 &= 6.34 \times 10^{8}, \\
  &\sigma_{zz} &= -p + \tau_{zz} =  6.34 \times 10^{8} + 6.34 \times 10^{8} &= 1.27 \times 10^{9}.
\end{eqnarray}

## Summary
| Stress components | $p=0$ at top &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;| $\sigma_{zz}=0$ at top &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;| $\sigma_{xx}=0$ at top &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;| ASPECT &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  |
|:----------- |-------------:|-----------------------:|-----------------------:|--------:|
| $\tau_{xx}$        | $-6.34\times10^{8}$ &nbsp; | $-6.34 \times 10^{8}$  | $-6.34 \times 10^{8}$ | $-6.34 \times 10^{8}$ |
| $\tau_{zz}$        | $6.34 \times 10^{8}$   | $6.34 \times 10^{8}$   | $6.34 \times 10^{8}$  | $6.34 \times 10^{8}$  |
| $p(z=0)$           | $1.29 \times 10^{10}$  | $1.36 \times 10^{10}$  | $1.23 \times 10^{10}$   | $1.36 \times 10^{10}$ |
| $p(z=H)$           | $0.0$                  | $6.34 \times 10^{8}$   | $-6.34 \times 10^{8}$  | $6.34 \times 10^{8}$  |
| $\sigma_{xx}(z=0)$ | $-1.35 \times 10^{10}$ | $-1.42 \times 10^{10}$ | $-1.29 \times 10^{10}$  | $1.29 \times 10^{10}$ |
| $\sigma_{xx}(z=H)$ | $-6.34 \times 10^{8}$  | $-1.27 \times 10^{9}$  |  $0$ | $0$            |
| $\sigma_{zz}(z=0)$ | $-1.23 \times 10^{10}$ | $-1.29 \times 10^{10}$ | $-1.17 \times 10^{10}$ | $1.42 \times 10^{10}$ |
| $\sigma_{zz}(z=H)$ | $6.34 \times 10^{8}$   | $0$                | $-1.27 \times 10^{9}$ | $1.27 \times 10^{9}$ |


The good news is that ASPECT's deviatoric stress values are the same with the expected values.

Pressure values from ASPECT are identical to our calculation for $\sigma_{zz}=0$, which is a reasonable assumption for the traction-free condition for the top surface.

However, ASPECT's full Cauchy stress values are different from any cases we've considered.
ASPECT's Cauchy stress values are retrieved as $p+\tau_{xx}$ and $p+\tau_{zz}$; or in general as $p\mathbf{I}+\boldsymbol{\tau}$. 

This way of constructing Cauchy stress seems plain wrong. It's not a matter of tension positive or compression positive: $p\mathbf{I}+\boldsymbol{\tau}$ is wrong either way. In the light of the opposite signs of pressure and deviatoric stresses in the governing equation, Cauchy stress should be $-p\mathbf{I}+\boldsymbol{\tau}$ (tension positive) or $p\mathbf{I}-\boldsymbol{\tau}$ (compression positive).
